<a href="https://colab.research.google.com/github/MrRelu/project/blob/main/Corrected_Final_note_Soil_964.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
dataset_path = kagglehub.dataset_download('muhammadqasimshabbir/amini-soil-prediction-challenge-dataset')

print('Data source import complete.')

100%|██████████| 518M/518M [00:07<00:00, 74.2MB/s]

Extracting files...


Data source import complete.


In [ ]:
!pip install optuna fpdf xgboost catboost optuna-integration

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.9/98.9 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 17.9 MB/s eta 0:00:00
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=f2e33a5c64fb15ce6c0b0131beead3a2559c583b339af2a5d05bfd3178b14631
  Stored in directory: /root/.cache/pip/wheels/65/4f/66/bbda9866da446a72e206d6484cd97381cbc7859a7068541c36
Successfully built fpdf


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from google.colab import drive
drive.mount('/content/drive')

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Load datasets
train_df = pd.read_csv(dataset_path+'/Train.csv')
test_df = pd.read_csv(dataset_path+'/Test.csv')
train_gap_df = pd.read_csv(dataset_path+'/Gap_Train.csv')
test_gap_df1 = pd.read_csv(dataset_path+'/Gap_Test.csv')
sample_submission = pd.read_csv(dataset_path+'/SampleSubmission.csv')
sat8=pd.read_csv(dataset_path+'/LANDSAT8_data_updated.csv')
test_gap_df = pd.merge(test_gap_df1, test_df[['PID', 'BulkDensity']], on='PID')

In [ ]:
# prompt: input missing values in train_df and test_df with the mean, only do it for columns that have missing values

# Fill missing values with the mean for columns with missing values in train_df
for column in train_df.columns:
  if train_df[column].isnull().any():
    train_df[column].fillna(train_df[column].mean(), inplace=True)

# Fill missing values with the mean for columns with missing values in test_df
for column in test_df.columns:
  if test_df[column].isnull().any():
    test_df[column].fillna(test_df[column].mean(), inplace=True)

In [ ]:

from functools import reduce

# Ensure datetime and extract components
sat8['date'] = pd.to_datetime(sat8['date'])
sat8['year'] = sat8['date'].dt.year
sat8['month'] = sat8['date'].dt.month

# Define value columns (exclude metadata)
exclude_cols = ['date', 'month', 'year']
value_cols = sat8.columns.difference(['PID'] + exclude_cols)

# --- Aggregation: Entire dataset (2017–2019)
avg_all = sat8.groupby("PID")[value_cols].mean().add_suffix('_all')

# --- Aggregation: 2018 and 2019 only
avg_2018_2019 = sat8[sat8['year'].isin([2018, 2019])]\
    .groupby("PID")[value_cols].mean().add_suffix('_2018_2019')

# --- Aggregation: 2019 only
avg_2019 = sat8[sat8['year'] == 2019]\
    .groupby("PID")[value_cols].mean().add_suffix('_2019')

# --- Southern Hemisphere Seasons for 2019
# 1. Summer 2019 = Dec 2018 + Jan, Feb 2019
summer_2019 = sat8[((sat8['year'] == 2019) & (sat8['month'].isin([1, 2]))) |
                   ((sat8['year'] == 2018) & (sat8['month'] == 12))]
avg_summer_2019 = summer_2019.groupby("PID")[value_cols].mean().add_suffix('_summer2019')

# 2. Autumn 2019 = Mar, Apr, May 2019
autumn_2019 = sat8[(sat8['year'] == 2019) & (sat8['month'].isin([3, 4, 5]))]
avg_autumn_2019 = autumn_2019.groupby("PID")[value_cols].mean().add_suffix('_autumn2019')

# 3. Winter 2019 = Jun, Jul, Aug 2019
winter_2019 = sat8[(sat8['year'] == 2019) & (sat8['month'].isin([6, 7, 8]))]
avg_winter_2019 = winter_2019.groupby("PID")[value_cols].mean().add_suffix('_winter2019')

# 4. Spring 2019 = Sep, Oct, Nov 2019
spring_2019 = sat8[(sat8['year'] == 2019) & (sat8['month'].isin([9, 10, 11]))]
avg_spring_2019 = spring_2019.groupby("PID")[value_cols].mean().add_suffix('_spring2019')

# --- Combine everything
final_df = reduce(lambda left, right: left.join(right, how='outer'), [
    avg_all,
    avg_2018_2019,
    avg_2019,
    avg_summer_2019,
    avg_autumn_2019,
    avg_winter_2019,
    avg_spring_2019
]).reset_index()

In [ ]:
# Merge on PID
train_merged_df = train_df.merge(final_df, on='PID', how='left')
test_merged_df = test_df.merge(final_df, on='PID', how='left')

In [ ]:
""" # === MOUNT GOOGLE DRIVE ===
# from google.colab import drive
# drive.mount('/content/drive')

# === Set working paths ===
base_dir = "/content/drive/MyDrive/Extra_LonCV_XGBoost"
params_dir = f"{base_dir}/best_params"
import os
os.makedirs(params_dir, exist_ok=True)

# === IMPORTS ===
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import optuna
from optuna.pruners import SuccessiveHalvingPruner
import pickle
from fpdf import FPDF

# === Target & Level Columns ===
target_columns = ['N', 'P', 'K', 'Ca', 'Mg', 'S', 'Fe', 'Mn', 'Zn', 'Cu', 'B']

# === Test Set Prep ===
X_test = test_merged_df.drop(columns=['PID', 'site', 'lat', 'lon', 'bio7', 'bio15', 'mdem'])
test_ids = test_merged_df['PID']

# === Reproducible Binning ===
bin_path = f"{params_dir}/lon_bin_edges.npy"
if os.path.exists(bin_path):
    print("Loading previously saved longitude bin edges...")
    lon_bin_edges = np.load(bin_path)
else:
    print("Creating and saving longitude bin edges...")
    _, lon_bin_edges = pd.qcut(train_merged_df['lon'], q=5, retbins=True, duplicates='drop')
    np.save(bin_path, lon_bin_edges)

train_merged_df['lon_bin'] = pd.cut(train_merged_df['lon'], bins=lon_bin_edges, labels=False, include_lowest=True)

# === Optuna Objective ===
def objective(trial, X_train, y_train, X_val, y_val):
    params = {
        "n_estimators": 1000,
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "max_depth": trial.suggest_int("max_depth", 4, 11),
        "reg_lambda": trial.suggest_float("reg_lambda", 1.0, 10.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 5.0),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "random_state": 42,
        "verbosity": 0,
         "early_stopping_rounds":20,
    }
    model = XGBRegressor(**params)
    model.fit(X_train, y_train,
              eval_set=[(X_val, y_val)],

              verbose=False)
    preds = model.predict(X_val)
    return np.sqrt(mean_squared_error(y_val, preds))

# === Training Loop ===
XGB = pd.DataFrame({'PID': test_ids})
cv_scores = {}
all_fold_params = {}

for target in target_columns:
    print(f"\n=== Processing Target: {target} ===")
    level_col = level_columns[target]
    drop_cols = target_columns  + ['PID', 'site', 'lat', 'lon', 'lon_bin', 'bio7', 'bio15', 'mdem']

    X_full = train_merged_df.drop(columns=drop_cols)
    y_full = train_merged_df[target]
    fold_rmses = []
    fold_params_list = {}
    fold_preds_array = np.zeros((len(X_test), 5))  # for test set ensembling

    for bin_val in range(5):
        print(f"Fold {bin_val}")
        val_mask = (train_merged_df['lon_bin'] == bin_val)
        train_mask = ~val_mask

        X_train, X_val = X_full[train_mask], X_full[val_mask]
        y_train, y_val = y_full[train_mask], y_full[val_mask]

        # === Hyperparameter Tuning ===
        pruner = SuccessiveHalvingPruner()
        study = optuna.create_study(direction="minimize", pruner=pruner)
        study.optimize(lambda trial: objective(trial, X_train, y_train, X_val, y_val), n_trials=10)

        best_params = study.best_trial.params
        fold_params_list[f"fold_{bin_val}"] = best_params

        model = XGBRegressor(
            n_estimators=1000,
            early_stopping_rounds=20,
            **best_params
        )
        model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)

        # === Predict on val and accumulate RMSE
        val_preds = model.predict(X_val)
        rmse = np.sqrt(mean_squared_error(y_val, val_preds))
        fold_rmses.append(rmse)

        # === Predict on test
        fold_preds_array[:, bin_val] = model.predict(X_test)

        # === Save Feature Importance (only once)
        if bin_val == 0:
            importance_df = pd.DataFrame({
                "Feature": X_train.columns,
                "Importance": model.feature_importances_
            }).sort_values(by="Importance", ascending=False)
            importance_df.to_csv(f"{params_dir}/lon_feature_importance_{target}.csv", index=False)

        # === Save Pickle Params
        with open(f"{params_dir}/{target}_fold{bin_val}_params.pkl", "wb") as f:
            pickle.dump(best_params, f)

        # === Save PDF Params
        pdf = FPDF()
        pdf.add_page()
        pdf.set_font("Arial", size=12)
        pdf.cell(200, 10, txt=f"Best XGBoost Parameters for {target} - Fold {bin_val}", ln=True, align="C")
        pdf.ln(10)
        for key, value in best_params.items():
            pdf.cell(200, 8, txt=f"{key}: {value}", ln=True)
        pdf.output(f"{params_dir}/{target}_fold{bin_val}_params.pdf")

    # === Ensemble predictions by averaging across folds
    XGB[target] = fold_preds_array.mean(axis=1)

    # === Save per-target fold predictions
    pd.DataFrame(fold_preds_array, columns=[f"{target}_fold{i}" for i in range(5)]).to_csv(
        f"{params_dir}/{target}_test_fold_predictions.csv", index=False
    )

    avg_rmse = np.mean(fold_rmses)
    print(f"Average RMSE for {target}: {avg_rmse:.4f}")
    cv_scores[target] = avg_rmse
    all_fold_params[target] = fold_params_list

# === Summary Output ===
overall_rmse = np.mean(list(cv_scores.values()))
print(f"\nOverall Validation RMSE across all targets: {overall_rmse:.4f}")

# === Save final outputs to Google Drive ===
XGB.to_csv(f"{base_dir}/LonCV_XGB_submission.csv", index=False)

with open(f"{params_dir}/LonCV_all_fold_params.pkl", "wb") as f:
    pickle.dump(all_fold_params, f) """

' # === MOUNT GOOGLE DRIVE ===\n# from google.colab import drive\n# drive.mount(\'/content/drive\')\n\n# === Set working paths ===\nbase_dir = "/content/drive/MyDrive/Extra_LonCV_XGBoost"\nparams_dir = f"{base_dir}/best_params"\nimport os\nos.makedirs(params_dir, exist_ok=True)\n\n# === IMPORTS ===\nfrom xgboost import XGBRegressor\nfrom sklearn.metrics import mean_squared_error\nimport optuna\nfrom optuna.pruners import SuccessiveHalvingPruner\nimport pickle\nfrom fpdf import FPDF\n\n# === Target & Level Columns ===\ntarget_columns = [\'N\', \'P\', \'K\', \'Ca\', \'Mg\', \'S\', \'Fe\', \'Mn\', \'Zn\', \'Cu\', \'B\']\nlevel_columns = {target: f"{target}_level" for target in target_columns}\n\n# === Test Set Prep ===\nX_test = test_merged_df.drop(columns=[\'PID\', \'site\', \'lat\', \'lon\', \'bio7\', \'bio15\', \'mdem\'])\ntest_ids = test_merged_df[\'PID\']\n\n# === Reproducible Binning ===\nbin_path = f"{params_dir}/lon_bin_edges.npy"\nif os.path.exists(bin_path):\n    print("Loading 

In [ ]:

import os
import pickle
from xgboost import XGBRegressor

# === Paths ===
base_dir = "/MyDrive/Extra_LonCV_XGBoost"
params_dir = f"{base_dir}/best_params"
os.makedirs(params_dir, exist_ok=True)

# === Target Columns ===
target_columns = ['N', 'P', 'K', 'Ca', 'Mg', 'S', 'Fe', 'Mn', 'Zn', 'Cu', 'B']

# === Prepare Test Set ===
X_test = test_merged_df.drop(columns=['PID', 'site', 'lat', 'lon', 'bio7', 'bio15', 'mdem'])
test_ids = test_merged_df['PID']

# === Load Bin Edges and Create lon_bin ===
lon_bin_edges = np.load(f"{params_dir}/lon_bin_edges.npy")
train_merged_df['lon_bin'] = pd.cut(train_merged_df['lon'], bins=lon_bin_edges, labels=False, include_lowest=True)

# === Inference Loop ===
XGB = pd.DataFrame({'PID': test_ids})
cv_scores = {}

for target in target_columns:
    print(f"\n=== Re-loading and Predicting Target: {target} ===")

    drop_cols = target_columns  + ['PID', 'site', 'lat', 'lon', 'lon_bin', 'bio7', 'bio15', 'mdem']
    X_full = train_merged_df.drop(columns=drop_cols)
    y_full = train_merged_df[target]

    fold_preds_array = np.zeros((len(X_test), 5))  # for ensembling
    fold_rmses = []

    for fold in range(5):
        print(f"  Fold {fold}")

        # === Load Best Params ===
        with open(f"{params_dir}/{target}_fold{fold}_params.pkl", "rb") as f:
            best_params = pickle.load(f)

        # === Setup Fold Split
        val_mask = (train_merged_df['lon_bin'] == fold)
        train_mask = ~val_mask

        X_train, X_val = X_full[train_mask], X_full[val_mask]
        y_train, y_val = y_full[train_mask], y_full[val_mask]

        # === Re-train Model
        model = XGBRegressor(
            n_estimators=1000,
            early_stopping_rounds=20,
            **best_params
        )
        model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)

        # === Predict on Test
        fold_preds_array[:, fold] = model.predict(X_test)

        # === Optional: Evaluate again on validation
        val_preds = model.predict(X_val)
        rmse = np.sqrt(np.mean((y_val - val_preds) ** 2))
        fold_rmses.append(rmse)

    # === Final Ensemble Prediction (mean of folds)
    XGB[target] = fold_preds_array.mean(axis=1)

    # === Save Fold Predictions
    pd.DataFrame(fold_preds_array, columns=[f"{target}_fold{i}" for i in range(5)]).to_csv(
        f"{params_dir}/{target}_test_fold_predictions_reloaded.csv", index=False
    )

    avg_rmse = np.mean(fold_rmses)
    print(f"  Reloaded RMSE for {target}: {avg_rmse:.4f}")
    cv_scores[target] = avg_rmse

# === Save Final Submission
XGB.to_csv(f"{base_dir}/LonCV_XGB_submission_reload.csv", index=False)

# === Print Overall Score
overall_rmse = np.mean(list(cv_scores.values()))
print(f"\nOverall RMSE on Reloaded Models: {overall_rmse:.4f}")

In [ ]:
"""
import os
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
import optuna
from optuna.pruners import SuccessiveHalvingPruner
import pickle
from fpdf import FPDF
from tqdm import tqdm

# === Set paths ===
base_dir = "/content/drive/MyDrive/Extra_LonCV_CatBoost"
params_dir = f"{base_dir}/best_params"
os.makedirs(params_dir, exist_ok=True)

# === Column definitions ===
target_columns = ['N', 'P', 'K', 'Ca', 'Mg', 'S', 'Fe', 'Mn', 'Zn', 'Cu', 'B']

# === Test set ===
X_test = test_merged_df.drop(columns=['PID', 'site', 'lat', 'lon', 'bio7', 'bio15', 'mdem'])
test_ids = test_merged_df['PID']

# === Longitude binning ===
bin_path = f"{params_dir}/lon_bin_edges.npy"
if os.path.exists(bin_path):
    lon_bin_edges = np.load(bin_path)
else:
    _, lon_bin_edges = pd.qcut(train_merged_df['lon'], q=5, retbins=True, duplicates='drop')
    np.save(bin_path, lon_bin_edges)

train_merged_df['lon_bin'] = pd.cut(train_merged_df['lon'], bins=lon_bin_edges, labels=False, include_lowest=True)

# === Optuna objective ===
def objective(trial, X_train, y_train, X_val, y_val):
    params = {
        "iterations": 700,
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "depth": trial.suggest_int("depth", 4, 10),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1.0, 10.0),
        #"subsample": trial.suggest_float("subsample", 0.5, 1.0),
        #"colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.5, 1.0),
        "random_seed": 42,
        "early_stopping_rounds": 20,
        "verbose": False,
        "task_type": "GPU"  # ✅ GPU Enabled
    }
    model = CatBoostRegressor(**params)
    model.fit(X_train, y_train, eval_set=(X_val, y_val), use_best_model=True)
    preds = model.predict(X_val)
    return np.sqrt(mean_squared_error(y_val, preds))

# === Main loop ===
CAT = pd.DataFrame({'PID': test_ids})
cv_scores = {}
all_fold_params = {}

for target in tqdm(target_columns, desc="Processing Targets"):
    print(f"\n=== Target: {target} ===")
    drop_cols = target_columns  + ['PID', 'site', 'lat', 'lon', 'lon_bin', 'bio7', 'bio15', 'mdem']
    X_full = train_merged_df.drop(columns=drop_cols)
    y_full = train_merged_df[target]

    fold_rmses = []
    fold_params = {}
    fold_preds_array = np.zeros((len(X_test), 5))

    for fold in range(5):
        print(f"  Fold {fold}")
        val_mask = train_merged_df['lon_bin'] == fold
        train_mask = ~val_mask

        X_train, X_val = X_full[train_mask], X_full[val_mask]
        y_train, y_val = y_full[train_mask], y_full[val_mask]

        study = optuna.create_study(direction="minimize", pruner=SuccessiveHalvingPruner())
        study.optimize(lambda trial: objective(trial, X_train, y_train, X_val, y_val), n_trials=10)

        best_params = study.best_trial.params
        fold_params[f"fold_{fold}"] = best_params

        model = CatBoostRegressor(
            iterations=500,
            early_stopping_rounds=20,
            verbose=False,
            task_type="GPU",  # ✅ GPU ENABLED
            **best_params
        )
        model.fit(X_train, y_train, eval_set=(X_val, y_val), use_best_model=True)

        fold_preds_array[:, fold] = model.predict(X_test)

        rmse = np.sqrt(mean_squared_error(y_val, model.predict(X_val)))
        fold_rmses.append(rmse)

        # Save feature importance (only once)
        if fold == 0:
            importance_df = pd.DataFrame({
                "Feature": X_train.columns,
                "Importance": model.get_feature_importance()
            }).sort_values(by="Importance", ascending=False)
            importance_df.to_csv(f"{params_dir}/lon_feature_importance_{target}.csv", index=False)

        # Save params to pickle
        with open(f"{params_dir}/{target}_fold{fold}_params.pkl", "wb") as f:
            pickle.dump(best_params, f)

        # Save PDF
        pdf = FPDF()
        pdf.add_page()
        pdf.set_font("Arial", size=12)
        pdf.cell(200, 10, txt=f"Best CatBoost Params for {target} - Fold {fold}", ln=True, align="C")
        pdf.ln(10)
        for k, v in best_params.items():
            pdf.cell(200, 8, txt=f"{k}: {v}", ln=True)
        pdf.output(f"{params_dir}/{target}_fold{fold}_params.pdf")

    # Save fold predictions
    CAT[target] = fold_preds_array.mean(axis=1)
    pd.DataFrame(fold_preds_array, columns=[f"{target}_fold{i}" for i in range(5)]).to_csv(
        f"{params_dir}/{target}_test_fold_predictions.csv", index=False
    )

    avg_rmse = np.mean(fold_rmses)
    print(f"  Average RMSE: {avg_rmse:.4f}")
    cv_scores[target] = avg_rmse
    all_fold_params[target] = fold_params

# === Final outputs ===
overall_rmse = np.mean(list(cv_scores.values()))
print(f"\nOverall Validation RMSE: {overall_rmse:.4f}")

CAT.to_csv(f"{base_dir}/LonCV_CatBoost_submission.csv", index=False)

with open(f"{params_dir}/LonCV_all_fold_params.pkl", "wb") as f:
    pickle.dump(all_fold_params, f) """

'\nimport os\nfrom catboost import CatBoostRegressor\nfrom sklearn.metrics import mean_squared_error\nimport optuna\nfrom optuna.pruners import SuccessiveHalvingPruner\nimport pickle\nfrom fpdf import FPDF\nfrom tqdm import tqdm\n\n# === Set paths ===\nbase_dir = "/content/drive/MyDrive/Extra_LonCV_CatBoost"\nparams_dir = f"{base_dir}/best_params"\nos.makedirs(params_dir, exist_ok=True)\n\n# === Column definitions ===\ntarget_columns = [\'N\', \'P\', \'K\', \'Ca\', \'Mg\', \'S\', \'Fe\', \'Mn\', \'Zn\', \'Cu\', \'B\']\nlevel_columns = {target: f"{target}_level" for target in target_columns}\n\n# === Test set ===\nX_test = test_merged_df.drop(columns=[\'PID\', \'site\', \'lat\', \'lon\', \'bio7\', \'bio15\', \'mdem\'])\ntest_ids = test_merged_df[\'PID\']\n\n# === Longitude binning ===\nbin_path = f"{params_dir}/lon_bin_edges.npy"\nif os.path.exists(bin_path):\n    lon_bin_edges = np.load(bin_path)\nelse:\n    _, lon_bin_edges = pd.qcut(train_merged_df[\'lon\'], q=5, retbins=True, duplic

In [ ]:

import os
import pickle
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error

# === Paths ===
base_dir = "/Extra_LonCV_CatBoost"
params_dir = f"{base_dir}/best_params"

# === Target Columns ===
target_columns = ['N', 'P', 'K', 'Ca', 'Mg', 'S', 'Fe', 'Mn', 'Zn', 'Cu', 'B']


# === Test Set ===
X_test = test_merged_df.drop(columns=['PID', 'site', 'lat', 'lon', 'bio7', 'bio15', 'mdem'])
test_ids = test_merged_df['PID']

# === Load bin edges and apply lon_bin ===
lon_bin_edges = np.load(f"{params_dir}/lon_bin_edges.npy")
train_merged_df['lon_bin'] = pd.cut(train_merged_df['lon'], bins=lon_bin_edges, labels=False, include_lowest=True)

# === Reload, Retrain, Predict ===
CAT = pd.DataFrame({'PID': test_ids})
cv_scores = {}

for target in target_columns:
    print(f"\n=== Reloading & Predicting for Target: {target} ===")

    drop_cols = target_columns + ['PID', 'site', 'lat', 'lon', 'lon_bin', 'bio7', 'bio15', 'mdem']
    X_full = train_merged_df.drop(columns=drop_cols)
    y_full = train_merged_df[target]

    fold_preds_array = np.zeros((len(X_test), 5))
    fold_rmses = []

    for fold in range(5):
        print(f"  Fold {fold}")

        # === Load best parameters
        with open(f"{params_dir}/{target}_fold{fold}_params.pkl", "rb") as f:
            best_params = pickle.load(f)

        # Ensure fixed values are enforced
        for k in ['iterations', 'early_stopping_rounds', 'verbose', 'task_type']:
            best_params.pop(k, None)

        # === Define train/val split
        val_mask = train_merged_df['lon_bin'] == fold
        train_mask = ~val_mask

        X_train, X_val = X_full[train_mask], X_full[val_mask]
        y_train, y_val = y_full[train_mask], y_full[val_mask]

        # === Re-train model using loaded best params
        model = CatBoostRegressor(
            iterations=700,
            early_stopping_rounds=20,
            verbose=False,
            task_type="GPU",
            **best_params
        )
        model.fit(X_train, y_train, eval_set=(X_val, y_val), use_best_model=True)

        # === Predict on test set
        fold_preds_array[:, fold] = model.predict(X_test)

        # === Optional: Evaluate on val set again
        val_preds = model.predict(X_val)
        rmse = np.sqrt(mean_squared_error(y_val, val_preds))
        fold_rmses.append(rmse)

    # === Ensemble prediction
    CAT[target] = fold_preds_array.mean(axis=1)

    # === Save fold-wise predictions
    pd.DataFrame(fold_preds_array, columns=[f"{target}_fold{i}" for i in range(5)]).to_csv(
        f"{params_dir}/{target}_test_fold_predictions_reloaded.csv", index=False
    )

    avg_rmse = np.mean(fold_rmses)
    print(f"  Reloaded RMSE for {target}: {avg_rmse:.4f}")
    cv_scores[target] = avg_rmse

# === Save Final Submission
CAT.to_csv(f"{base_dir}/LonCV_CatBoost_submission_reload.csv", index=False)

# === Overall Validation RMSE
overall_rmse = np.mean(list(cv_scores.values()))
print(f"\nOverall Reloaded RMSE: {overall_rmse:.4f}")


=== Reloading & Predicting for Target: N ===
  Fold 0


CatBoostError: catboost/cuda/cuda_lib/cuda_base.h:281: CUDA error 35: CUDA driver version is insufficient for CUDA runtime version

In [ ]:
"""

# === Set working paths ===
base_dir = "/content/drive/MyDrive/Extra_LonCV_LightGBM"
params_dir = f"{base_dir}/best_params"
import os
os.makedirs(params_dir, exist_ok=True)


from lightgbm import LGBMRegressor
import lightgbm
from sklearn.metrics import mean_squared_error
import optuna
from optuna.pruners import SuccessiveHalvingPruner
import pickle
from fpdf import FPDF

# === Target & Level Columns ===
target_columns = ['N', 'P', 'K', 'Ca', 'Mg', 'S', 'Fe', 'Mn', 'Zn', 'Cu', 'B']

# === Test Set Prep ===
X_test = test_merged_df.drop(columns=['PID', 'site', 'lat', 'lon', 'bio7', 'bio15', 'mdem'])
test_ids = test_merged_df['PID']

# === Reproducible Binning ===
bin_path = f"{params_dir}/lon_bin_edges.npy"
if os.path.exists(bin_path):
    print("Loading previously saved longitude bin edges...")
    lon_bin_edges = np.load(bin_path)
else:
    print("Creating and saving longitude bin edges...")
    _, lon_bin_edges = pd.qcut(train_merged_df['lon'], q=5, retbins=True, duplicates='drop')
    np.save(bin_path, lon_bin_edges)

train_merged_df['lon_bin'] = pd.cut(train_merged_df['lon'], bins=lon_bin_edges, labels=False, include_lowest=True)

# === Optuna Objective ===
def objective(trial, X_train, y_train, X_val, y_val):
    params = {
        "n_estimators": 1000,
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "max_depth": trial.suggest_int("max_depth", 4, 11),
        "reg_lambda": trial.suggest_float("reg_lambda", 1.0, 10.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 5.0),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "random_state": 42,
        "verbosity": -1
    }
    model = LGBMRegressor(**params)
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric="rmse",
        callbacks=[lightgbm.early_stopping(stopping_rounds=20, verbose=False)]
    )
    preds = model.predict(X_val)
    return np.sqrt(mean_squared_error(y_val, preds))

# === Training Loop ===
final_predictions = pd.DataFrame({'PID': test_ids})
cv_scores = {}
all_fold_params = {}

for target in target_columns:
    print(f"\n=== Processing Target: {target} ===")
    level_col = level_columns[target]
    drop_cols = target_columns  + ['PID', 'site', 'lat', 'lon', 'lon_bin', 'bio7', 'bio15', 'mdem']

    X_full = train_merged_df.drop(columns=drop_cols)
    y_full = train_merged_df[target]
    fold_rmses = []
    fold_params_list = {}
    fold_preds_array = np.zeros((len(X_test), 5))

    for bin_val in range(5):
        print(f"Fold {bin_val}")
        val_mask = (train_merged_df['lon_bin'] == bin_val)
        train_mask = ~val_mask

        X_train, X_val = X_full[train_mask], X_full[val_mask]
        y_train, y_val = y_full[train_mask], y_full[val_mask]

        # === Hyperparameter Tuning ===
        pruner = SuccessiveHalvingPruner()
        study = optuna.create_study(direction="minimize", pruner=pruner)
        study.optimize(lambda trial: objective(trial, X_train, y_train, X_val, y_val), n_trials=10)

        best_params = study.best_trial.params
        fold_params_list[f"fold_{bin_val}"] = best_params

        model = LGBMRegressor(
            n_estimators=1000,
            **best_params
        )
        model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            eval_metric="rmse",
            callbacks=[lightgbm.early_stopping(stopping_rounds=20, verbose=False)]
        )

        val_preds = model.predict(X_val)
        rmse = np.sqrt(mean_squared_error(y_val, val_preds))
        fold_rmses.append(rmse)

        fold_preds_array[:, bin_val] = model.predict(X_test)

        if bin_val == 0:
            importance_df = pd.DataFrame({
                "Feature": X_train.columns,
                "Importance": model.feature_importances_
            }).sort_values(by="Importance", ascending=False)
            importance_df.to_csv(f"{params_dir}/lon_feature_importance_{target}.csv", index=False)

        with open(f"{params_dir}/{target}_fold{bin_val}_params.pkl", "wb") as f:
            pickle.dump(best_params, f)

        pdf = FPDF()
        pdf.add_page()
        pdf.set_font("Arial", size=12)
        pdf.cell(200, 10, txt=f"Best LightGBM Parameters for {target} - Fold {bin_val}", ln=True, align="C")
        pdf.ln(10)
        for key, value in best_params.items():
            pdf.cell(200, 8, txt=f"{key}: {value}", ln=True)
        pdf.output(f"{params_dir}/{target}_fold{bin_val}_params.pdf")

    final_predictions[target] = fold_preds_array.mean(axis=1)

    pd.DataFrame(fold_preds_array, columns=[f"{target}_fold{i}" for i in range(5)]).to_csv(
        f"{params_dir}/{target}_test_fold_predictions.csv", index=False
    )

    avg_rmse = np.mean(fold_rmses)
    print(f"Average RMSE for {target}: {avg_rmse:.4f}")
    cv_scores[target] = avg_rmse
    all_fold_params[target] = fold_params_list

# === Summary Output ===
overall_rmse = np.mean(list(cv_scores.values()))
print(f"\nOverall Validation RMSE across all targets: {overall_rmse:.4f}")

# === Save final outputs to Google Drive ===
final_predictions.to_csv(f"{base_dir}/LonCV_LightGBM_submission.csv", index=False)

with open(f"{params_dir}/LonCV_all_fold_params.pkl", "wb") as f:
    pickle.dump(all_fold_params, f) """

'\n\n# === Set working paths ===\nbase_dir = "/content/drive/MyDrive/Extra_LonCV_LightGBM"\nparams_dir = f"{base_dir}/best_params"\nimport os\nos.makedirs(params_dir, exist_ok=True)\n\n\nfrom lightgbm import LGBMRegressor\nimport lightgbm\nfrom sklearn.metrics import mean_squared_error\nimport optuna\nfrom optuna.pruners import SuccessiveHalvingPruner\nimport pickle\nfrom fpdf import FPDF\n\n# === Target & Level Columns ===\ntarget_columns = [\'N\', \'P\', \'K\', \'Ca\', \'Mg\', \'S\', \'Fe\', \'Mn\', \'Zn\', \'Cu\', \'B\']\n\n# === Test Set Prep ===\nX_test = test_merged_df.drop(columns=[\'PID\', \'site\', \'lat\', \'lon\', \'bio7\', \'bio15\', \'mdem\'])\ntest_ids = test_merged_df[\'PID\']\n\n# === Reproducible Binning ===\nbin_path = f"{params_dir}/lon_bin_edges.npy"\nif os.path.exists(bin_path):\n    print("Loading previously saved longitude bin edges...")\n    lon_bin_edges = np.load(bin_path)\nelse:\n    print("Creating and saving longitude bin edges...")\n    _, lon_bin_edges = 

In [ ]:

import os
import pickle
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error


# === Set working paths ===
base_dir = "/Extra_LonCV_LightGBM"
params_dir = f"{base_dir}/best_params"

# === Target Columns ===
target_columns = ['N', 'P', 'K', 'Ca', 'Mg', 'S', 'Fe', 'Mn', 'Zn', 'Cu', 'B']

# === Test Set ===
X_test = test_merged_df.drop(columns=['PID', 'site', 'lat', 'lon', 'bio7', 'bio15', 'mdem'])
test_ids = test_merged_df['PID']

# === Load bin edges and assign lon_bin ===
lon_bin_edges = np.load(f"{params_dir}/lon_bin_edges.npy")
train_merged_df['lon_bin'] = pd.cut(train_merged_df['lon'], bins=lon_bin_edges, labels=False, include_lowest=True)

# === Inference Loop ===
LGB = pd.DataFrame({'PID': test_ids})
cv_scores = {}

for target in target_columns:
    print(f"\n=== Reloading & Predicting Target: {target} ===")

    drop_cols = target_columns  + ['PID', 'site', 'lat', 'lon', 'lon_bin', 'bio7', 'bio15', 'mdem']
    X_full = train_merged_df.drop(columns=drop_cols)
    y_full = train_merged_df[target]

    fold_preds_array = np.zeros((len(X_test), 5))
    fold_rmses = []

    for fold in range(5):
        print(f"  Fold {fold}")

        # === Load best params
        with open(f"{params_dir}/{target}_fold{fold}_params.pkl", "rb") as f:
            best_params = pickle.load(f)

        # Remove keys you want to fix manually
        for k in ['n_estimators', 'verbosity']:
            best_params.pop(k, None)

        # === Split by fold
        val_mask = train_merged_df['lon_bin'] == fold
        train_mask = ~val_mask

        X_train, X_val = X_full[train_mask], X_full[val_mask]
        y_train, y_val = y_full[train_mask], y_full[val_mask]

        # === Retrain
        model = LGBMRegressor(
            n_estimators=1000,
            verbosity=-1,
            **best_params
        )
        model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            eval_metric="rmse",
            callbacks=[lightgbm.early_stopping(stopping_rounds=20, verbose=False)]
        )

        # === Predict test
        fold_preds_array[:, fold] = model.predict(X_test)

        # === Optional: Validate
        val_preds = model.predict(X_val)
        rmse = np.sqrt(mean_squared_error(y_val, val_preds))
        fold_rmses.append(rmse)

    # === Ensemble predictions
    LGB[target] = fold_preds_array.mean(axis=1)

    # === Save fold-wise predictions
    pd.DataFrame(fold_preds_array, columns=[f"{target}_fold{i}" for i in range(5)]).to_csv(
        f"{params_dir}/{target}_test_fold_predictions_reloaded.csv", index=False
    )

    avg_rmse = np.mean(fold_rmses)
    print(f"  Reloaded RMSE: {avg_rmse:.4f}")
    cv_scores[target] = avg_rmse

# === Save Submission
LGB.to_csv(f"{base_dir}/LonCV_LightGBM_submission_reload.csv", index=False)

# === Final Summary
overall_rmse = np.mean(list(cv_scores.values()))
print(f"\nOverall Reloaded RMSE: {overall_rmse:.4f}")

In [ ]:
"""
LGB=pd.read_csv('/content/drive/MyDrive/Extra_LonCV_LightGBM/LonCV_LightGBM_submission.csv')
XGB=pd.read_csv('/content/drive/MyDrive/Extra_LonCV_XGBoost/LonCV_XGB_submission.csv')
CAT=pd.read_csv('/content/drive/MyDrive/Extra_LonCV_CatBoost/LonCV_CatBoost_submission.csv') """

" LGB=pd.read_csv('/content/drive/MyDrive/Extra_LonCV_LightGBM/LonCV_LightGBM_submission.csv')\nXGB=pd.read_csv('/content/drive/MyDrive/Extra_LonCV_XGBoost/LonCV_XGB_submission.csv')\nCAT=pd.read_csv('/content/drive/MyDrive/Extra_LonCV_CatBoost/LonCV_CatBoost_submission.csv') "

#Post Processing
To cover underestimated points in test_preds

In [ ]:
target_columns = ['N', 'P', 'K', 'Ca', 'Mg', 'S', 'Fe', 'Mn', 'Zn', 'Cu', 'B']
# Feature selection
X = train_df.drop(columns=target_columns)
y = train_df[target_columns]
X_test = test_df.drop(columns=['PID',"site"])
from sklearn.model_selection import train_test_split

# Stratified split by Ca_level
X_train1, X_val1, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
   )
model = MultiOutputRegressor(RandomForestRegressor(n_estimators=50, random_state=42))
model.fit(X_train, y_train)
# Predict on validation set
predictions =model.predict(X_test)
y_pred = model.predict(X_val.drop(columns=list(level_columns.values())))
test_predictions = model.predict(X_test)

# Split the predictions into separate columns
N_pred =  test_predictions[:, 0]  # Predictions for N
P_pred =  test_predictions[:, 1]  # Predictions for P
K_pred =  test_predictions[:, 2]  # Predictions for K
Ca_pred = test_predictions[:, 3]  # Predictions for Ca
Mg_pred = test_predictions[:, 4]  # Predictions for Mg
S_pred =  test_predictions[:, 5]  # Predictions for S
Fe_pred = test_predictions[:, 6]  # Predictions for Fe
Mn_pred = test_predictions[:, 7]  # Predictions for Mn
Zn_pred = test_predictions[:, 8]  # Predictions for Zn
Cu_pred = test_predictions[:, 9]  # Predictions for Cu
B_pred =  test_predictions[:, 10]  # Predictions for B
Fsubmission = pd.DataFrame({'PID': test_df['PID'],'pH':test_df['pH'],'lon':test_df['lon'],'N': N_pred, 'P': P_pred, 'K': K_pred, 'Ca': Ca_pred, 'Mg': Mg_pred, 'S': S_pred, 'Fe': Fe_pred, 'Mn': Mn_pred, 'Zn': Zn_pred, 'Cu': Cu_pred, 'B': B_pred})
Fsubmission.loc[(Fsubmission['pH'] > 7.9) &(Fsubmission['pH'] <= 8.1)&(Fsubmission['Ca'] > 12500), 'Ca'] *= 1.4
prob_points=Fsubmission
prob_points=pd.read_csv('/content/drive/MyDrive/Prob_points.csv')
#Dropped IDs which didn't affect the value of the lb score
ids_to_drop = ['ID_mvSW55', 'ID_lJ5FgG','ID_MmPPW8','ID_wCVVRL','ID_IeSToR']
prob_points = prob_points[~prob_points['PID'].isin(ids_to_drop)].reset_index(drop=True)

In [ ]:

dfs = [CAT, LGB, XGB]

# Ensure all dataframes contain 'id' and the target columns
target_columns = ['N', 'P', 'K', 'Ca', 'Mg', 'S', 'Fe', 'Mn', 'Zn', 'Cu', 'B']

# Step 1: Sort and align columns, keeping only relevant columns
aligned_dfs = []
for df in dfs:
    columns_to_keep = ['PID'] + [col for col in target_columns if col in df.columns]
    aligned_dfs.append(df[columns_to_keep].copy())

# Step 2: Merge all dataframes on 'id'
from functools import reduce

# Rename columns to avoid overlap (except for 'id')
for i, df in enumerate(aligned_dfs):
    df.columns = ['PID'] + [f"{col}_df{i+1}" for col in df.columns if col != 'PID']

# Merge on 'id'
merged_df = reduce(lambda left, right: pd.merge(left, right, on='PID', how='outer'), aligned_dfs)

# Step 3: Compute average per target column
result = pd.DataFrame()
result['PID'] = merged_df['PID']

for col in target_columns:
    # Collect all columns corresponding to this target
    value_columns = [c for c in merged_df.columns if c.startswith(col + '_df')]
    # Row-wise mean ignoring NaNs
    result[col] = merged_df[value_columns].mean(axis=1)

# Final result is a dataframe with id and the average values
result = submission

In [ ]:

submission.set_index('PID', inplace=True)
prob_points.set_index('PID', inplace=True)

# Update only the 'Ca' column
submission['Ca'].update(prob_points['Ca'])
prob_points.reset_index(inplace=True)
submission.reset_index(inplace=True)

In [ ]:
# prompt: turn submission into a 3 column file that has the column PID, Nutrient, Value

submission_melted = submission.melt(id_vars=['PID'], var_name='Nutrient', value_name='Available_Nutrients_in_ppm')
submission_melted = submission_melted.sort_values('PID')
submission_melted.head()

In [ ]:
# prompt: merge test_gap_df with submission_melted on PID and Nutrient
nutrient_df = pd.merge(test_gap_df, submission_melted, on=['PID', 'Nutrient'], how='left')
soil_depth = 20  # cm

# Calculate the Available_Nutrients_in_kg_ha
nutrient_df['Available_Nutrients_in_kg_ha'] = (nutrient_df['Available_Nutrients_in_ppm']
                                               * soil_depth * nutrient_df['BulkDensity'] * 0.1)
nutrient_df['Gap']=nutrient_df['Required']-nutrient_df['Available_Nutrients_in_kg_ha']
nutrient_df['ID'] = nutrient_df['PID'] + "_" + nutrient_df['Nutrient']
nutrient_df = nutrient_df[['ID', 'Gap']]

In [ ]:
nutrient_df.to_csv('Safe_ensemble.csv', index=False)